# **IND320 Project Log** – Assignment 4: Machine Learning

**Esteban Carrasco**
*November 20, 2025*

---

## **1. Project Overview**
This project focused on analyzing **hourly electricity production and consumption data** from Elhub for Norwegian price areas. Using Python (Pandas, NumPy, SciPy, Plotly), the goal was to develop interactive visualizations and predictive models to explore relationships between meteorological conditions and energy dynamics.

The dataset, provided in CSV format, required preprocessing to handle temporal variations and diverse production types (hydro, wind, etc.). The analysis was deployed as a **Streamlit web application** for user-friendly exploration.

**Links**
- **Streamlit App**: [ind320-projectwork-esteban-carrasco.streamlit.app](https://ind320-projectwork-esteban-carrasco.streamlit.app)
- **GitHub Repository**: [github.com/Ficus22/IND320-ProjectWork](https://github.com/Ficus22/IND320-ProjectWork)

---

## **2. Development Process**

### **2.1 Data Handling and Objectives**
The project extended the Streamlit app with new features, including interactive maps, sliding window correlation analysis, and dynamic forecasting.

Data from **2021–2024** was retrieved via the Elhub API for both production (`PRODUCTION_PER_GROUP_MBA_HOUR`) and consumption (`CONSUMPTION_PER_GROUP_MBA_HOUR`). Due to storage constraints, Cassandra was not used, and MongoDB was the primary database for storing and querying the data.

Key objectives included:
- Implementing **interactive map visualizations** with GeoJSON overlays for Norwegian price areas (NO1–NO5).
- Developing **snow drift calculations** and visualizations per year and month.
- Enabling **meteorology-energy correlation analysis** using sliding windows.
- Adding **SARIMAX-based forecasting** for energy production and consumption.
- Improving user experience with caching, spinners, and intuitive page organization.

#### **Challenges and Solutions**
The main challenge was **storage limitations**, which prevented the use of Cassandra. Instead, MongoDB was used to store and manage the dataset efficiently. Collaboration with classmates helped validate STL decomposition parameters and frequency analysis approaches.

### **2.2 AI Assistance**
*Le Chat* ([Mistral AI](https://mistral.ai)) provided support in:
- Designing the **interactive map interface** with Plotly and Folium.
- Integrating **snow drift calculations** into Streamlit with dynamic plotting.
- Converting the **sliding window correlation** analysis into an interactive Streamlit module.
- Implementing the **SARIMAX forecasting interface** and visualizations.
- Explaining SARIMAX parameters and translating the project documentation into English.

---

## **3. Streamlit Phase**

### **Features Added**
The Streamlit app was enhanced with several key features:

**Interactive Map & Selectors**
A Plotly/Folium-based map with GeoJSON overlays allows users to select price areas, view coordinates, and analyze production/consumption groups over custom time intervals. The map highlights selected areas and stores user-clicked coordinates for further analysis. Choropleth coloring reflects mean energy values, improving interpretability.

**Snow Drift Calculation & Visualization**
Snow drift was calculated from **July 1 to June 30** of the following year to align with seasonal patterns. Users can select a range of years and visualize results with wind roses and monthly plots.

**Meteorology & Energy Production Correlation**
A **sliding window correlation** tool was integrated to explore how meteorological variables (e.g., temperature, wind speed) influence energy production. Users can adjust lag and window length to detect delayed effects and compare correlations during normal vs. extreme weather conditions.

The following plots illustrate the relationship between **temperature and energy production** over time:

1. **Lag Effect Scan**
   ![Lag Effect Scan Plot](LagPlot.png)
   The plot shows how correlation varies across different lags, revealing cyclical patterns that align with daily and seasonal cycles. Positive peaks indicate periods where temperature changes strongly influence energy production, while negative troughs suggest inverse relationships.

2. **Aligned Time-Series (Normalized)**
   ![Aligned Time-Series (Normalized) Plot](AllignedPlot.png)
   This plot displays normalized temperature and energy production from **2021 to 2024**, highlighting seasonal trends and short-term fluctuations. The alignment of spikes and dips suggests a dynamic relationship, where temperature changes often precede shifts in energy production.

**Discussion on Correlation Over Time**
The sliding window analysis reveals that temperature and energy production exhibit time-dependent correlations. During warmer months, higher temperatures often correlate with increased energy production (e.g., solar output), while colder periods may show inverse trends due to reduced efficiency or demand shifts. The lag scan further confirms that temperature impacts energy production with a delay of several hours, likely due to system response times. These insights underscore the importance of time-lagged modeling in energy forecasting.

**Forecasting Energy Production & Consumption**
A dynamic SARIMAX interface allows users to define training periods, forecast horizons, and exogenous variables (e.g., weather data). The model provides interactive forecasts with confidence intervals, making it accessible for both experts and non-experts.

**UX Improvements**
Pages were reordered for better workflow, and caching was implemented to speed up computations. Spinners were added to indicate progress during long calculations, enhancing the overall user experience.

### **Challenges and Adaptations**
- Ensured smooth **map interactions** with coordinate selections and area highlighting.
- Adjusted **snow drift visualizations** for dynamic year and month selection.
- Maintained **consistent color scales** across choropleth maps for clarity.
- Validated **meteorological and production dataset compatibility** for accurate correlation analysis.
- Simplified the **forecasting interface** with input validation to accommodate non-expert users.

---

## **4. Jupyter Notebook Phase**

For html rndering

In [1]:
%matplotlib inline

In [2]:
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

## Elhub API

Testing API connection

In [4]:
import requests

entity ="price-areas"
dataset = "PRODUCTION_PER_GROUP_MBA_HOUR"
URL = f"https://api.elhub.no/energy-data/v0/{entity}?dataset={dataset}"
response = requests.get(URL)

print(response.status_code)

200


Fetching data from Elhub API

In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
import pandas as pd

ENTITY = "price-areas"
DATASET_PRODUCTION = "PRODUCTION_PER_GROUP_MBA_HOUR"
YEARS_PRODUCTION = [2022, 2023, 2024]

def generate_monthly_ranges(year):
    start_year = datetime(year, 1, 1)
    end_year = datetime(year, 12, 31)
    ranges = []
    current = start_year
    while current <= end_year:
        month_start = current
        month_end = (current + relativedelta(months=1)) - relativedelta(seconds=1)
        start_str = month_start.strftime("%Y-%m-%dT%H:%M:%S") + "%2B01:00"
        end_str = month_end.strftime("%Y-%m-%dT%H:%M:%S") + "%2B01:00"
        ranges.append((start_str, end_str))
        current += relativedelta(months=1)
    return ranges

all_production_records = []

for year in YEARS_PRODUCTION:
    monthly_ranges = generate_monthly_ranges(year)
    for start_date, end_date in monthly_ranges:
        url = f"https://api.elhub.no/energy-data/v0/{ENTITY}?dataset={DATASET_PRODUCTION}&startDate={start_date}&endDate={end_date}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            for entry in data.get("data", []):
                records = entry.get("attributes", {}).get("productionPerGroupMbaHour", [])
                all_production_records.extend(records)
            #print(f"Fetched {len(records)} records for {start_date[:10]} ({year}).")
        else:
            print(f"Error {response.status_code} for {start_date[:10]} ({year}).")

print(f"Total production records fetched (2022-2024): {len(all_production_records)}")


Total production records fetched (2022-2024): 657600


In [5]:
# display the head
for i, record in enumerate(all_production_records[:5]):
    print(f"Record {i+1}:")
    print(record)
    print("-" * 50)

Record 1:
{'endTime': '2022-01-01T01:00:00+01:00', 'lastUpdatedTime': '2025-02-01T18:02:57+01:00', 'priceArea': 'NO1', 'productionGroup': 'hydro', 'quantityKwh': 1291422.4, 'startTime': '2022-01-01T00:00:00+01:00'}
--------------------------------------------------
Record 2:
{'endTime': '2022-01-01T02:00:00+01:00', 'lastUpdatedTime': '2025-02-01T18:02:57+01:00', 'priceArea': 'NO1', 'productionGroup': 'hydro', 'quantityKwh': 1246209.4, 'startTime': '2022-01-01T01:00:00+01:00'}
--------------------------------------------------
Record 3:
{'endTime': '2022-01-01T03:00:00+01:00', 'lastUpdatedTime': '2025-02-01T18:02:57+01:00', 'priceArea': 'NO1', 'productionGroup': 'hydro', 'quantityKwh': 1271757.0, 'startTime': '2022-01-01T02:00:00+01:00'}
--------------------------------------------------
Record 4:
{'endTime': '2022-01-01T04:00:00+01:00', 'lastUpdatedTime': '2025-02-01T18:02:57+01:00', 'priceArea': 'NO1', 'productionGroup': 'hydro', 'quantityKwh': 1204251.8, 'startTime': '2022-01-01T03:0

## Adding data into MongoDB

Connection test

In [6]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get the URI from environment variables
uri = os.getenv("MONGO_URI")

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


Transform API records to match existing collection format using a dataframe to be faster

In [4]:
import pandas as pd

# Créer un DataFrame directement depuis la liste de dicts
df = pd.DataFrame(all_production_records)

#print(df.head())

# Rename columns and keep only the necessary ones
df = df.rename(columns={
    "priceArea": "price_area",
    "productionGroup": "production_group",
    "startTime": "start_time",
    "quantityKwh": "quantity_kwh"
})[["price_area", "production_group", "start_time", "quantity_kwh"]]

df["start_time"] = pd.to_datetime(df["start_time"].str[:-6])


print(df.dtypes)
print(df.head())

price_area                  object
production_group            object
start_time          datetime64[ns]
quantity_kwh               float64
dtype: object
  price_area production_group          start_time  quantity_kwh
0        NO1            hydro 2022-01-01 00:00:00     1291422.4
1        NO1            hydro 2022-01-01 01:00:00     1246209.4
2        NO1            hydro 2022-01-01 02:00:00     1271757.0
3        NO1            hydro 2022-01-01 03:00:00     1204251.8
4        NO1            hydro 2022-01-01 04:00:00     1202086.9


In [5]:
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
import os
import pandas as pd

# Connect to MongoDB
load_dotenv()
client = MongoClient(os.getenv("MONGO_URI"), server_api=ServerApi('1'))
db = client["elhub_data"]
collection = db["production_data"]

# Convert the DataFrame to a dictionary and insert
data_for_mongo = df.to_dict("records")
collection.insert_many(data_for_mongo)
print(f"{len(data_for_mongo)} documents inserted into MongoDB.")


657600 documents inserted into MongoDB.


## Adding data into Cassandra table

Connect to Cassandra

In [8]:
from cassandra.cluster import Cluster
from cassandra.concurrent import execute_concurrent_with_args
import sys

KEYSPACE = "my_ind320_keyspace"
TABLE = "elhub_data"
CASSANDRA_HOST = "localhost"
CASSANDRA_PORT = 9042
CONCURRENCY_LEVEL = 100  # number of simultaneous inserts

try:
    cluster = Cluster([CASSANDRA_HOST], port=CASSANDRA_PORT)
    session = cluster.connect()
    print("✅ Connected to Cassandra.")
except Exception as e:
    sys.exit(f"❌ Cassandra connection error: {e}")

✅ Connected to Cassandra.


Connection test

In [9]:
session.set_keyspace('my_ind320_keyspace')
rows = session.execute("SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'my_ind320_keyspace';")
for row in rows:
    print(row.table_name)

elhub_data
my_first_table


In [12]:
df_cassandra = df

print(df_cassandra.dtypes)
print(df_cassandra.head())
print(len(df_cassandra))

price_area                  object
production_group            object
start_time          datetime64[ns]
quantity_kwh               float64
dtype: object
  price_area production_group          start_time  quantity_kwh
0        NO1            hydro 2022-01-01 00:00:00     1291422.4
1        NO1            hydro 2022-01-01 01:00:00     1246209.4
2        NO1            hydro 2022-01-01 02:00:00     1271757.0
3        NO1            hydro 2022-01-01 03:00:00     1204251.8
4        NO1            hydro 2022-01-01 04:00:00     1202086.9
657600


Import data

In [ ]:
from tqdm import tqdm

session.set_keyspace(KEYSPACE)

insert_query = session.prepare(f"""
    INSERT INTO {TABLE} (price_area, production_group, start_time, quantity_kwh)
    VALUES (?, ?, ?, ?)
""")


print("🚀 Inserting data into Cassandra...")

params = [
    (
        row["price_area"],
        row["production_group"],
        row["start_time"].to_pydatetime(),
        float(row["quantity_kwh"])
    )
    for _, row in df_cassandra.iterrows()
]

# Use tqdm for progress tracking
results = list(
    tqdm(
        execute_concurrent_with_args(
            session, insert_query, params, concurrency=CONCURRENCY_LEVEL
        ),
        total=len(params),
        desc="Insertion"
    )
)

# Check for potential errors
errors = [res for res in results if not res[0]]
if errors:
    print(f"⚠️ {len(errors)} insertion errors detected.")
else:
    print("✅ All data inserted successfully!")


cluster.shutdown()
print("\n🏁 Import completed.")

🚀 Inserting data into Cassandra...


WriteTimeout: Error from server: code=1100 [Coordinator node timed out waiting for replica nodes' responses] message="Operation timed out - received only 0 responses." info={'consistency': 'LOCAL_ONE', 'required_responses': 1, 'received_responses': 0, 'write_type': 'SIMPLE'}

## Adding a new table into databases

Fetching data from Elhub API

In [6]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
import pandas as pd

# CONSTANTS FOR CONSUMPTION
ENTITY = "price-areas"
DATASET_CONSUMPTION = "CONSUMPTION_PER_GROUP_MBA_HOUR"
YEARS_CONSUMPTION = [2021, 2022, 2023, 2024]


# FUNCTION TO GENERATE MONTHLY DATE RANGES
def generate_monthly_ranges(year):
    start_year = datetime(year, 1, 1)
    end_year = datetime(year, 12, 31)
    ranges = []
    current = start_year
    while current <= end_year:
        month_start = current
        month_end = (current + relativedelta(months=1)) - relativedelta(seconds=1)
        # Formatting for URL encoding +01:00
        start_str = month_start.strftime("%Y-%m-%dT%H:%M:%S") + "%2B01:00"
        end_str = month_end.strftime("%Y-%m-%dT%H:%M:%S") + "%2B01:00"
        ranges.append((start_str, end_str))
        current += relativedelta(months=1)
    return ranges


# FETCH HOURLY CONSUMPTION DATA 2021 - 2024
all_consumption_records = []

for year in YEARS_CONSUMPTION:
    monthly_ranges = generate_monthly_ranges(year)
    for start_date, end_date in monthly_ranges:
        url = f"https://api.elhub.no/energy-data/v0/{ENTITY}?dataset={DATASET_CONSUMPTION}&startDate={start_date}&endDate={end_date}"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            for entry in data.get("data", []):
                records = entry.get("attributes", {}).get("consumptionPerGroupMbaHour", [])
                all_consumption_records.extend(records)
        else:
            print(f"Error {response.status_code} for {start_date[:10]} ({year}).")

print(f"Total consumption records fetched (2021–2024): {len(all_consumption_records)}")


Total consumption records fetched (2021–2024): 876600


In [7]:
# display the head
for i, record in enumerate(all_consumption_records[:5]):
    print(f"Record {i+1}:")
    print(record)
    print("-" * 50)

Record 1:
{'consumptionGroup': 'cabin', 'endTime': '2021-01-01T01:00:00+01:00', 'lastUpdatedTime': '2024-12-20T10:35:40+01:00', 'meteringPointCount': 100607, 'priceArea': 'NO1', 'quantityKwh': 177071.56, 'startTime': '2021-01-01T00:00:00+01:00'}
--------------------------------------------------
Record 2:
{'consumptionGroup': 'cabin', 'endTime': '2021-01-01T02:00:00+01:00', 'lastUpdatedTime': '2024-12-20T10:35:40+01:00', 'meteringPointCount': 100607, 'priceArea': 'NO1', 'quantityKwh': 171335.12, 'startTime': '2021-01-01T01:00:00+01:00'}
--------------------------------------------------
Record 3:
{'consumptionGroup': 'cabin', 'endTime': '2021-01-01T03:00:00+01:00', 'lastUpdatedTime': '2024-12-20T10:35:40+01:00', 'meteringPointCount': 100607, 'priceArea': 'NO1', 'quantityKwh': 164912.02, 'startTime': '2021-01-01T02:00:00+01:00'}
--------------------------------------------------
Record 4:
{'consumptionGroup': 'cabin', 'endTime': '2021-01-01T04:00:00+01:00', 'lastUpdatedTime': '2024-12-2

#### In MongoDB

In [ ]:
import pandas as pd

df_cons = pd.DataFrame(all_consumption_records)

# Keep only needed fields and rename to target format
df_cons = df_cons.rename(columns={
    "priceArea": "price_area",
    "consumptionGroup": "consumption_group",
    "startTime": "start_time",
    "quantityKwh": "quantity_kwh",
    "meteringPointCount": "metering_point_count"
})[[
    "price_area",
    "consumption_group",
    "start_time",
    "quantity_kwh",
    "metering_point_count"
]]

# Convert start_time to datetime
df_cons["start_time"] = pd.to_datetime(df_cons["start_time"], utc=True)

print(df_cons.dtypes)
print(df_cons.head())

C:\Users\Esteban\AppData\Local\Temp\ipykernel_10180\2678956536.py:21: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_cons["start_time"] = pd.to_datetime(df_cons["start_time"])


price_area               object
consumption_group        object
start_time               object
quantity_kwh            float64
metering_point_count      int64
dtype: object
  price_area consumption_group                 start_time  quantity_kwh  \
0        NO1             cabin  2021-01-01 00:00:00+01:00     177071.56   
1        NO1             cabin  2021-01-01 01:00:00+01:00     171335.12   
2        NO1             cabin  2021-01-01 02:00:00+01:00     164912.02   
3        NO1             cabin  2021-01-01 03:00:00+01:00     160265.77   
4        NO1             cabin  2021-01-01 04:00:00+01:00     159828.69   

   metering_point_count  
0                100607  
1                100607  
2                100607  
3                100607  
4                100607  


In [9]:
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
import os

load_dotenv()
client = MongoClient(os.getenv("MONGO_URI"), server_api=ServerApi('1'))
db = client["elhub_data"]
collection = db["consumption_data"]  # NEW TABLE (COLLECTION)

# Transform DataFrame to list of dicts and insert
data_for_mongo = df_cons.to_dict("records")
collection.insert_many(data_for_mongo)

print(f"{len(data_for_mongo)} hourly consumption records inserted (2021–2024).")

876600 hourly consumption records inserted (2021–2024).


#### In Cassandra